<a href="https://colab.research.google.com/github/PTC-Education/PTC-Education.github.io/blob/master/UR3e_Onshape_Monitor_Digital_Twin_with_ThingWorx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install onshape-client
from onshape_client.client import Client
import json
base = 'https://cad.onshape.com' # change this if you're using a document in an enterprise (i.e. "https://ptc.onshape.com")

In [ ]:
import os
def include(filename):
    if os.path.exists(filename): 
        execfile(filename)
include('colabkeys.py')
client = Client(configuration={"base_url": base,
                               "access_key": access,
                               "secret_key": secret})

# Get UR3 Data from TWX

Data from the physical robot is being sent to ThingWorx through Kepware, so we are pulling the joint data from ThingWorx REST API

In [ ]:
import requests
import json

def getUR3Data():
  appKey = twkey
  URL = 'https://pp-2101111403aw.portal.ptc.io/Thingworx/Things/UR_Robot1/Properties'

  headers = {
          'appKey': appKey,
          'accept': 'application/json',
          'Content-Type':'application/json'
      }

  response = requests.get(URL,headers=headers)
  DiceValuePos = [response.json()['rows'][0]['lastRollValue'],response.json()['rows'][0]['DiceDemo_dx'],response.json()['rows'][0]['DiceDemo_dy'],response.json()['rows'][0]['gripperState'],response.json()['rows'][0]['readytocount'],response.json()['rows'][0]['DiceDemo_drz'],response.json()['rows'][0]['UR_Robots_UR_Robot1_TCP_TCP-rx']]
  RobotData = [response.json()['rows'][0]['joint1Angle'],response.json()['rows'][0]['joint2Angle'],response.json()['rows'][0]['joint3Angle'],response.json()['rows'][0]['joint4Angle'],response.json()['rows'][0]['joint5Angle'],response.json()['rows'][0]['joint6Angle'],response.json()['rows'][0]['gripperState']*0.015]
  return DiceValuePos, RobotData

In [ ]:
print(getUR3Data())

([3, 119, 352, 1, 1, 103, 20], [-43.89918442632478, -96.95507625543905, -94.72054085442886, -79.02149726784427, 90.87110630774858, 45.6074404924135, 1])


# LEGO Brick Color Update

Find Part ID of LEGO Brick

In [ ]:
# The first part of the fixed URL specifies which API we're calling
fixed_url = '/api/parts/d/did/w/wid'

# https://cad.onshape.com/documents/f45fad4854ca95357c3937ee/w/947c0b3bc5c329a1c2960014/e/560babcca9702c62ed1034af
did = 'f45fad4854ca95357c3937ee'
wid = '947c0b3bc5c329a1c2960014'

# Method below specifies the request type - 'GET' or 'POST'
method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# The command below prints the entire JSON response from Onshape
# print(json.loads(response.data))

parts = json.loads(response.data)
for i in range(len(parts)):
  print(parts[i]["name"] +" has part ID: "+ parts[i]["partId"])

Set "partMetaData" variable to the meta data from LEGO brick part

In [ ]:
# https://cad.onshape.com/documents/f45fad4854ca95357c3937ee/w/947c0b3bc5c329a1c2960014/e/560babcca9702c62ed1034af
fixed_url = '/api/metadata/d/did/w/wid/e/eid/p/pid'
did = 'f45fad4854ca95357c3937ee'
wid = '947c0b3bc5c329a1c2960014'
eid = '560babcca9702c62ed1034af'
pid = 'JLD'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
          'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)
fixed_url = fixed_url.replace('pid', pid)

response = client.api_client.request('GET', url=base + fixed_url,
query_params=params, headers=headers, body=payload)

partMetaData = json.loads(response.data)

function for changing color of LEGO brick

In [ ]:
def updateColor(partMetaData,newColors):
  partProperties = partMetaData['properties']

  # # Print current appearance values
  # for x in partProperties:
  #   if x['name'] == "Appearance":
  #     partAppearance = x
  #     break
  # print(partAppearance['value'])

  for i in range(len(partProperties)):
    if partProperties[i]['name'] == "Appearance":
      partProperties[i]['value']['color'] = newColors
      break

  partMetaData['properties'] = partProperties

  payload = partMetaData

  response = client.api_client.request('POST', url=base + fixed_url,
  query_params=params, headers=headers, body=payload)

In [ ]:
updateColor(partMetaData,{'red': 1, 'green': 255, 'blue': 255})

# Update Dice Number Text

Load FeatureInfo Variable

In [ ]:
fixed_url = '/api/partstudios/d/did/w/wid/e/eid/features'

#https://cad.onshape.com/documents/f45fad4854ca95357c3937ee/w/947c0b3bc5c329a1c2960014/e/600c5bc17d1d06585c85b9e8
did = 'f45fad4854ca95357c3937ee'
wid = '947c0b3bc5c329a1c2960014'
eid = '600c5bc17d1d06585c85b9e8'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

method = 'GET'
params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
          'Content-Type': 'application/json'}

FeatureInfo = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

In [ ]:
def updateValue(value):
  newFeature = json.loads(FeatureInfo.data)["features"][0]
  newFeature["message"]["entities"][0]["message"]['parameters'][1]['message']['value'] = '''Last Roll: '''+value
  # print(newFeature["message"]["entities"][0]["message"]['parameters'][1]['message']['value'])
  # print(timeRemaining)
  # print(json.loads(response.data)["sourceMicroversion"])
  # print(json.loads(response.data)["serializationVersion"])

  fixed_url = '/api/partstudios/d/did/w/wid/e/eid/features/featureid/fid'

  fid = json.loads(FeatureInfo.data)["features"][0]["message"]["featureId"]

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid, 1)
  fixed_url = fixed_url.replace('fid', fid)

  # print(fixed_url)

  method = 'POST'
  params = {}
  payload = {'feature': newFeature,
            'serializationVersion': json.loads(FeatureInfo.data)["serializationVersion"],
            'sourceMicroversion': json.loads(FeatureInfo.data)["sourceMicroversion"]}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

In [ ]:
updateValue('5')

# Gripper Open/Close

In [ ]:
fixed_url = '/api/elements/d/did/w/wid/e/eid/configuration'

# https://cad.onshape.com/documents/f45fad4854ca95357c3937ee/w/947c0b3bc5c329a1c2960014/e/e13b8f1b2de53390ea9d2e34
did = 'f45fad4854ca95357c3937ee'
wid = '947c0b3bc5c329a1c2960014'
eid = '6361bbad5f20ace895955cf6'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

# Method below specifies the request type - 'GET' or 'POST'
method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
gripperConfigInfo = json.loads(response.data)
# ConfigParams = ConfigInfo["configurationParameters"]

# for i in range(len(ConfigParams)):
#   print(ConfigParams[i]["message"]["parameterName"])
#   print(ConfigParams[i]["message"]["parameterId"])

In [ ]:
def moveGripper(openclose):
  fixed_url = '/api/elements/d/did/w/wid/e/eid/configuration'

  # https://cad.onshape.com/documents/f45fad4854ca95357c3937ee/w/947c0b3bc5c329a1c2960014/e/e13b8f1b2de53390ea9d2e34
  did = 'f45fad4854ca95357c3937ee'
  wid = '947c0b3bc5c329a1c2960014'
  eid = '6361bbad5f20ace895955cf6'

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  # Method below specifies the request type - 'GET' or 'POST'
  method = 'POST'
  ConfigParams = gripperConfigInfo["configurationParameters"]
  if openclose == 'open':
    ConfigParams[0]["message"]["rangeAndDefault"]["message"]["minValue"] = 0
    ConfigParams[0]["message"]["rangeAndDefault"]["message"]["maxValue"] = 0
    ConfigParams[0]["message"]["rangeAndDefault"]["message"]["defaultValue"] = 0
  else:
    ConfigParams[0]["message"]["rangeAndDefault"]["message"]["minValue"] = 0.016
    ConfigParams[0]["message"]["rangeAndDefault"]["message"]["maxValue"] = 0.016
    ConfigParams[0]["message"]["rangeAndDefault"]["message"]["defaultValue"] = 0.016

  params = {}
  payload = {'configurationParameters': ConfigParams,
            'serializationVersion': ConfigInfo["serializationVersion"],
            'sourceMicroversion': ConfigInfo["sourceMicroversion"]}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

In [ ]:
moveGripper('open')

# Matrix Library

In [ ]:
import math
import numpy

def DiceTransform(value):
    switcher = {
        0: [
            0, 0, 1, 0,
            0, 1, 0, -0.4,
            -1, 0, 0, -0.015,
            0, 0, 0, 1
           ],
        1: [
            0, 0, 1, 0,
            0, 1, 0, -0.4,
            -1, 0, 0, -0.015,
            0, 0, 0, 1
           ],
        2: [
            1, 0, 0, 0,
            0, 0, 1, -0.4,
            0, -1, 0, -0.015,
            0, 0, 0, 1
           ],
        3: [
            1, 0, 0, 0,
            0, -1, 0, -0.4,
            0, 0, -1, -0.015,
            0, 0, 0, 1
           ],
        4: [
            1, 0, 0, 0,
            0, 1, 0, -0.4,
            0, 0, 1, -0.015,
            0, 0, 0, 1
           ],
        5: [
            1, 0, 0, 0,
            0, 0, -1, -0.4,
            0, 1, 0, -0.015,
            0, 0, 0, 1
           ],
        6: [
            0, 0, -1, 0,
            0, 1, 0, -0.4,
            1, 0, 0, -0.015,
            0, 0, 0, 1
           ] 
    }
  
    # get() method of dictionary data type returns 
    # value of passed argument if it is present 
    # in dictionary otherwise second argument will
    # be assigned as default value of passed argument
    return switcher.get(value, "nothing")

def DiceMoveSpin(value,x1,y1):
  matrix = DiceTransform(value)
  matrix[3] = x1
  matrix[7] = y1
  return matrix

def DiceFollow(diceMatrix,tcpMatrix):
  diceMatrix[3] = tcpMatrix[3]
  diceMatrix[7] = tcpMatrix[7]
  diceMatrix[11] = tcpMatrix[11]
  return diceMatrix
  
def sixteenToFourByFour(matrix):
  fourbyfour = [[matrix[0],matrix[1],matrix[2],matrix[3]],
                [matrix[4],matrix[5],matrix[6],matrix[7]],
                [matrix[8],matrix[9],matrix[10],matrix[11]],
                [matrix[12],matrix[13],matrix[14],matrix[15]]]
  return fourbyfour

def fourByFourToSixteen(matrix):
  sixteen = [matrix[0][0],matrix[0][1],matrix[0][2],matrix[0][3],
             matrix[1][0],matrix[1][1],matrix[1][2],matrix[1][3],
             matrix[2][0],matrix[2][1],matrix[2][2],matrix[2][3],
             matrix[3][0],matrix[3][1],matrix[3][2],matrix[3][3]]
  return sixteen

def clockwiseSpinZ(theta):
  m = [[math.cos(theta), math.sin(theta), 0, 0],
       [-math.sin(theta), math.cos(theta), 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]]
  return m

def clockwiseSpinX(theta):
  m = [[1, 0, 0, 0],
       [0, math.cos(theta), math.sin(theta), 0],
       [0, -math.sin(theta), math.cos(theta), 0],
       [0, 0, 0, 1]
       ]
  return m


# Dice Control

In [ ]:
fixed_url = '/api/assemblies/d/did/w/wid/e/eid'

# https://cad.onshape.com/documents/f45fad4854ca95357c3937ee/w/947c0b3bc5c329a1c2960014/e/e13b8f1b2de53390ea9d2e34
did = 'f45fad4854ca95357c3937ee'
wid = '947c0b3bc5c329a1c2960014'
eid = 'e13b8f1b2de53390ea9d2e34'

method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
# print(response.data)
# The command below prints the entire JSON response from Onshape
occurances = json.loads(response.data)['rootAssembly']['occurrences']
for x in occurances:
  # print(x['path'])
  if x['path'][0] == 'MyTMZjC8CDrpoJtvv':
    diceId = x
  elif x['path'][0] == 'MPcb81Pj8AzXubNwt':
    tcpId = x

In [ ]:
import numpy
def DiceControl(following,value,dicex,dicey,thetaz,thetax,thetay):
  ## Get assembly occurances
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid'

  # https://cad.onshape.com/documents/f45fad4854ca95357c3937ee/w/947c0b3bc5c329a1c2960014/e/e13b8f1b2de53390ea9d2e34
  did = 'f45fad4854ca95357c3937ee'
  wid = '947c0b3bc5c329a1c2960014'
  eid = 'e13b8f1b2de53390ea9d2e34'

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  occurances = json.loads(response.data)['rootAssembly']['occurrences']

  for x in occurances:
    if x['path'][0] == 'MyTMZjC8CDrpoJtvv':
      diceOccurence = x
    elif x['path'][0] == 'MPcb81Pj8AzXubNwt':
      tcpId = x


  ## Send assembly occurence transforms
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/occurrencetransforms'

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  method = 'POST'

  params = {}

  if following == True:
    payload = {'isRelative':False,
              'occurrences':[diceOccurence],
              'transform': fourByFourToSixteen(numpy.matmul(sixteenToFourByFour(DiceFollow(DiceMoveSpin(value,0.2,-0.4),tcpId['transform'])),clockwiseSpinX(thetax)))
        }
  else:
    # payload = {'isRelative':False,
    #           'occurrences':[diceOccurence],
    #           'transform': DiceMoveSpin(getDiceValue()[0],dicex,dicey)
    #     }
    payload = {'isRelative':False,
              'occurrences':[diceOccurence],
               'transform': DiceMoveSpin(value,dicex,dicey)
              # 'transform': fourByFourToSixteen(numpy.matmul(sixteenToFourByFour(DiceMoveSpin(value,dicex,dicey)),clockwiseSpinZ(thetaz)))
        }

  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)


In [ ]:
DiceControl(False,3,.2,-.4,0,0.1,0)

In [ ]:
import numpy
print(fourByFourToSixteen(numpy.matmul(sixteenToFourByFour(DiceMoveSpin(4,1,2)),clockwiseSpinZ(math.pi/2))))

[6.123233995736766e-17, 1.0, 0.0, 1.0, -1.0, 6.123233995736766e-17, 0.0, 2.0, 0.0, 0.0, 1.0, -0.015, 0.0, 0.0, 0.0, 1.0]


# Update Joint Configurations

In [ ]:
fixed_url = '/api/elements/d/did/w/wid/e/eid/configuration'

# https://cad.onshape.com/documents/f45fad4854ca95357c3937ee/w/947c0b3bc5c329a1c2960014/e/e13b8f1b2de53390ea9d2e34
did = 'f45fad4854ca95357c3937ee'
wid = '947c0b3bc5c329a1c2960014'
eid = 'e13b8f1b2de53390ea9d2e34'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

# Method below specifies the request type - 'GET' or 'POST'
method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
ConfigInfo = json.loads(response.data)
ConfigParams = ConfigInfo["configurationParameters"]

for i in range(len(ConfigParams)):
  print(ConfigParams[i]["message"]["parameterName"])
  print(ConfigParams[i]["message"]["parameterId"])
  # print(ConfigParams[i]["message"]["rangeAndDefault"]["message"]) # useful info if configurations are variables
  

In [ ]:
def updateJoints(jointArray):
  fixed_url = '/api/elements/d/did/w/wid/e/eid/configuration'

  # https://cad.onshape.com/documents/f45fad4854ca95357c3937ee/w/947c0b3bc5c329a1c2960014/e/e13b8f1b2de53390ea9d2e34
  did = 'f45fad4854ca95357c3937ee'
  wid = '947c0b3bc5c329a1c2960014'
  eid = 'e13b8f1b2de53390ea9d2e34'

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  # Method below specifies the request type - 'GET' or 'POST'
  method = 'POST'

  ConfigParams = ConfigInfo["configurationParameters"]
  for i in range(len(ConfigParams)):
    ConfigParams[i]["message"]["rangeAndDefault"]["message"]["minValue"] = jointArray[i]
    ConfigParams[i]["message"]["rangeAndDefault"]["message"]["maxValue"] = jointArray[i]
    ConfigParams[i]["message"]["rangeAndDefault"]["message"]["defaultValue"] = jointArray[i]


  params = {}
  payload = {'configurationParameters': ConfigParams,
            'serializationVersion': ConfigInfo["serializationVersion"],
            'sourceMicroversion': ConfigInfo["sourceMicroversion"]}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)


# Update Joint Mate Values

In [ ]:
RobotData = [0.1,-0.5,0,0,0,0,0]

fixed_url = '/api/assemblies/d/did/w/wid/e/eid/matevalues'

# https://cad.onshape.com/documents/f45fad4854ca95357c3937ee/w/947c0b3bc5c329a1c2960014/e/32946638068e1d1428bac606
did = 'f45fad4854ca95357c3937ee'
wid = '947c0b3bc5c329a1c2960014'
eid = '32946638068e1d1428bac606'

method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v2+json',
           'Content-Type': 'application/vnd.onshape.v2+json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
fullResponse = json.loads(response.data)

for i in range(len(fullResponse["mateValues"])):
  if fullResponse['mateValues'][i]['mateName'] == "Base revolve":
    fullResponse['mateValues'][i]['rotationZ'] = RobotData[0]
  elif fullResponse['mateValues'][i]['mateName'] == "Shoulder revolve":
    fullResponse['mateValues'][i]['rotationZ'] = RobotData[1]
  elif fullResponse['mateValues'][i]['mateName'] == "Elbow revolve":
    fullResponse['mateValues'][i]['rotationZ'] = RobotData[2]
  elif fullResponse['mateValues'][i]['mateName'] == "Wrist 1 revolve":
    fullResponse['mateValues'][i]['rotationZ'] = RobotData[3]
  elif fullResponse['mateValues'][i]['mateName'] == "Wrist 2 revolve":
    fullResponse['mateValues'][i]['rotationZ'] = RobotData[4]
  elif fullResponse['mateValues'][i]['mateName'] == "Wrist 3 revolve":
    fullResponse['mateValues'][i]['rotationZ'] = RobotData[5]
  elif fullResponse['mateValues'][i]['mateName'] == "Gripper 1":
    fullResponse['mateValues'][i]['translationZ'] = RobotData[6]

method = 'POST'

params = {}
payload = fullResponse
headers = {'Accept': 'application/vnd.onshape.v2+json',
           'Content-Type': 'application/vnd.onshape.v2+json'}
response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# The command below prints the entire JSON response from Onshape
print(response.status)

200


# Main Loop

In [ ]:
import time
while True:
  # updateValue(str(getDiceValue()[0]))
  time.sleep(0.25)
  dicevalarray, UR3array = getUR3Data()
  try:
    updateJoints(UR3array)
    if dicevalarray[4] == 1:
      DiceControl(dicevalarray[3],dicevalarray[0],0.143,-0.312,0,0,0)
    else:
      DiceControl(dicevalarray[3],dicevalarray[0],dicevalarray[1]*0.001+0.16,dicevalarray[2]*-0.001,0,0,0)
  except:
    print('fail')